In [5]:
import argparse

from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np

import tensorflow as tf

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, concatenate
from tensorflow.keras.optimizers import Adam

In [6]:
# pre-define about column info
COLUMNS = ["age", "workclass", "fnlwgt", "education", "education_num", "marital_status",
          "occupation", "relationship", "race", "gender", "capital_gain", "capital_loss", "hours_per_week", "native_country", "income_bracket"]
LABEL_COLUMN = "label"
CATEGORICAL_COLUMNS = ["workclass", "education", "marital_status", "occupation", "relationship", "race", "gender", "native_country"]
NUMERICAL_COLUMNS = ["age", "education_num", "capital_gain", "capital_loss", "hours_per_week"]

In [7]:
# class Deep:
#     def __init__(self, args):
#         self.learning_rate = args.learning_rate
#         self.epochs = args.epochs
#         self.batch_size = args.batch_size
#         self.input_dim = args.input_dim
#         self.model = self.classifier()

#     def classifier(self):
#         model = Sequential()
#         model.add(Dense(100, activation='relu', input_dim=self.input_dim))
#         model.add(Dense(50, activation='relu'))
#         model.add(Dense(1, activation='sigmoid'))

#         optimizer = Adam(lr=self.learning_rate, beta_1=0.9, beta_2=0.999)
#         model.compile(loss='binary_crossentropy',
#                       optimizer=optimizer,
#                       metrics=['accuracy'])
#         return model

#     def fit(self, x, y):
#         self.model.fit(x, y, epochs=self.epochs, batch_size=self.batch_size, validation_split=0.2)

#     def print_performance(self, x, y):
#         performance_test = self.model.evaluate(x, y, batch_size=self.batch_size)
#         print('Test Loss and Accuracy ->', performance_test)
        
# class Wide:
#     def __init__(self, args):
#         self.learning_rate = args.learning_rate
#         self.epochs = args.epochs
#         self.batch_size = args.batch_size
#         self.input_dim = args.input_dim
#         self.model = self.classifier()

#     def classifier(self):
#         model = Sequential()
#         model.add(Dense(1, activation='sigmoid', input_dim=self.input_dim))

#         optimizer = Adam(lr=self.learning_rate, beta_1=0.9, beta_2=0.999)
#         model.compile(loss='binary_crossentropy',
#                       optimizer=optimizer,
#                       metrics=['accuracy'])
#         return model

#     def fit(self, x, y):
#         self.model.fit(x, y, epochs=self.epochs, batch_size=self.batch_size, validation_split=0.2)

#     def print_performance(self, x, y):
#         performance_test = self.model.evaluate(x, y, batch_size=self.batch_size)
#         print('Test Loss and Accuracy ->', performance_test)

In [8]:
class WideAndDeep:
    def __init__(self, batch_size, learning_rate, epochs, input_dim):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size
        self.input_dim = input_dim
        self.model = self.classifier()

    def classifier(self):
        optimizer = Adam(lr=self.learning_rate, beta_1=0.9, beta_2=0.999)

        # wide part
        wide = Input(shape=(self.input_dim,))

        # deep part
        deep_input = Input(shape=(self.input_dim,))
        deep = Dense(100, activation='relu')(deep_input)
        deep = Dense(50, activation='relu')(deep)

        # concatenate : wide and deep
        wide_n_deep = concatenate([wide, deep])
        wide_n_deep = Dense(1, activation='sigmoid')(wide_n_deep)
        model = Model(inputs=[wide, deep_input], outputs=wide_n_deep)
        model.compile(loss='binary_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])
        return model

    def fit(self, wide_x, deep_x, y):
        self.model.fit([wide_x, deep_x], y, epochs=self.epochs, batch_size=self.batch_size, validation_split=0.2)

    def print_performance(self, wide_x, deep_x, y):
        performance_test = self.model.evaluate([wide_x, deep_x], y, batch_size=self.batch_size)
        print('Test Loss and Accuracy ->', performance_test)

In [9]:
# def main(model_param):
# prepare dataset
df_train = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data',
                       header=None,
                       names=COLUMNS,
                       skipinitialspace=True)
df_test = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test',
                      header=None,
                      names=COLUMNS,
                      skipinitialspace=True,
                      skiprows=1)

In [16]:
df = pd.concat([df_train, df_test])
df[LABEL_COLUMN] = df['income_bracket'].apply(lambda x: ">50K" in x).astype(int)
y = df[LABEL_COLUMN].values
df.pop(LABEL_COLUMN)
df.pop("income_bracket")
df = pd.get_dummies(df, columns=[x for x in CATEGORICAL_COLUMNS])

In [18]:
df = pd.DataFrame(MinMaxScaler().fit_transform(df), columns=df.columns)
x = df.values

# split train, test
train_length = len(df_train)
x_train = x[:train_length]
y_train = y[:train_length]
x_test = x[train_length:]
y_test = y[train_length:]

In [20]:
x_train

array([[0.30136986, 0.04413121, 0.8       , ..., 1.        , 0.        ,
        0.        ],
       [0.45205479, 0.04805174, 0.8       , ..., 1.        , 0.        ,
        0.        ],
       [0.28767123, 0.13758131, 0.53333333, ..., 1.        , 0.        ,
        0.        ],
       ...,
       [0.56164384, 0.09446153, 0.53333333, ..., 1.        , 0.        ,
        0.        ],
       [0.06849315, 0.12800425, 0.53333333, ..., 1.        , 0.        ,
        0.        ],
       [0.47945205, 0.18648211, 0.53333333, ..., 1.        , 0.        ,
        0.        ]])

In [21]:
    # prepare hyper parameter
#     parser = argparse.ArgumentParser()
#     parser.add_argument('--batch_size', type=int, default=500,
#                         help='Batch size for networks')
#     parser.add_argument('--epochs', type=int, default=30,
#                         help='Epochs for the networks')
#     parser.add_argument('--learning_rate', type=float, default=0.001,
#                         help='Learning rate')
#     parser.add_argument('--input_dim', type=int, default=x_train.shape[1],
#                         help='Input dimension for the generator.')
#     args = parser.parse_args()

#     if model_param == "deep":
#         deep = Deep(args)
#         deep.fit(x_train, y_train)
#         deep.print_performance(x_test, y_test)
#     elif model_param == 'wide':
#         wide = Wide(args)
#         wide.fit(x_train, y_train)
#         wide.print_performance(x_test, y_test)
#     else:
wide_n_deep = WideAndDeep(batch_size = 500,
                          learning_rate = 0.001,
                         epochs = 30,
                         input_dim = x_train.shape[1])
wide_n_deep.fit(x_train, x_train, y_train)
wide_n_deep.print_performance(x_test, x_test, y_test)

Epoch 1/30
53/53 [==============================] - 0s 8ms/step - loss: 0.4804 - accuracy: 0.7682 - val_loss: 0.3971 - val_accuracy: 0.8119
Epoch 2/30
53/53 [==============================] - 0s 7ms/step - loss: 0.3714 - accuracy: 0.8287 - val_loss: 0.3553 - val_accuracy: 0.8346
Epoch 3/30
53/53 [==============================] - 0s 5ms/step - loss: 0.3489 - accuracy: 0.8365 - val_loss: 0.3440 - val_accuracy: 0.8397
Epoch 4/30
53/53 [==============================] - 0s 5ms/step - loss: 0.3393 - accuracy: 0.8399 - val_loss: 0.3369 - val_accuracy: 0.8437
Epoch 5/30
53/53 [==============================] - 0s 5ms/step - loss: 0.3345 - accuracy: 0.8451 - val_loss: 0.3333 - val_accuracy: 0.8446
Epoch 6/30
53/53 [==============================] - 0s 5ms/step - loss: 0.3274 - accuracy: 0.8467 - val_loss: 0.3325 - val_accuracy: 0.8460
Epoch 7/30
53/53 [==============================] - 0s 5ms/step - loss: 0.3238 - accuracy: 0.8488 - val_loss: 0.3316 - val_accuracy: 0.8452
Epoch 8/30
53/53 [==

In [27]:
# prediction for individual
x_predict_test = x_test[np.newaxis, 444, :]
y_predict_test = y_test[444]
result = wide_n_deep.model.predict([x_predict_test, x_predict_test])
print('result predicted:', result[0][0])
print('result predicted:', round(result[0][0]))
print('result real:', y_predict_test)


# if __name__ == '__main__':
#     main('widendeep')

result predicted: 5.501997e-05
result predicted: 0.0
result real: 0


In [ ]:
x_pred = x_test[np.newaxis, :, :]
y_pred = 